# Performing landscape level predictions

Mount drive, install package


In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install git+https://github.com/PatBall1/detectree2.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Cloning https://github.com/PatBall1/detectree2.git to /tmp/pip-req-build-qfxcvqqq
  Running command git clone --filter=blob:none --quiet https://github.com/PatBall1/detectree2.git /tmp/pip-req-build-qfxcvqqq
  Resolved https://github.com/PatBall1/detectree2.git to commit e640838223bbb136c9475b01eef9162659ba19a5
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-install-31840k44/detectron2_a5752b94174c4a769216532d29d8d229
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-31840k44/detectron2_a5752b94174c4a769216532d29d8d229
  Resolved https://github.com/facebookresearch/detectron2.git to commit b15f64ec4429e23a148972175a0207c5a9ab84cf
  Preparing metadata (setup.py) ... done


## Tiling
Tile up the entire orthomosaic to make predictions

In [2]:
import rasterio
###
site_path = "/content/drive/MyDrive/Sukute"

img_path = site_path + "/rgb/odm_orthophoto.tif"
#img_path = site_path + "/rgb/2016/Paracou_RGB_2016_10cm.tif"
tiles_path = site_path + "/tilespred_map10/"
#model_path = "/content/drive/Shareddrives/detectree2/models/220629_ParacouSepilokDanum_JB.pth"

# Read in the tiff file
data = rasterio.open(img_path)


In [4]:
from detectree2.preprocessing.tiling import tile_data
buffer = 20
tile_width = 45
tile_height = 45
tile_data(img_path, tiles_path, buffer, tile_width, tile_height, dtype_bool = True)

  0%|          | 0/72 [00:00<?, ?it/s]

## Create full site predictions

### Access pre-trained model

Download pre-trained model

In [8]:
#!wget https://github.com/PatBall1/detectree2/raw/master/model_garden/230103_randresize_full.pth

!wget https://zenodo.org/records/15014353/files/230103_randresize_full.pth

--2025-06-28 10:08:28--  https://zenodo.org/records/15014353/files/230103_randresize_full.pth
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.48.194, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 498209001 (475M) [application/octet-stream]
Saving to: ‘230103_randresize_full.pth’

230103_randresize_f 100%[===================>] 475.13M  22.0MB/s    in 24s     

2025-06-28 10:08:53 (20.0 MB/s) - ‘230103_randresize_full.pth’ saved [498209001/498209001]



There might be a cleaner way to do it with `pkg_resources` but I haven't figured this out yet

In [13]:
from detectree2.models.train import MyTrainer, setup_cfg

# pretrained model?
#trained_model = "/content/drive/Shareddrives/detectree2/models/220629_ParacouSepilokDanum_JB.pth"
#trained_model = "/content/drive/Shareddrives/detectree2/models/220723_withParacouUAV.pth"
#trained_model = "/content/drive/Shareddrives/detectree2/models/230103_resize_full/model_4.pth"
trained_model = "./230103_randresize_full.pth"

cfg = setup_cfg(update_model=trained_model) # update_model arg can be used to load in trained  model

Make predictions!

In [15]:
from detectree2.models.predict import predict_on_data
from detectron2.engine import DefaultPredictor
#cfg.MODEL.DEVICE='cpu'
predict_on_data(tiles_path, tiles_path + "predict_on_data/", DefaultPredictor(cfg))

Predicting 61 files in mode rgb


/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Predicted 50 files of 61


Here we reproject predictions into geographical space

In [17]:
from detectree2.models.outputs import project_to_geojson
project_to_geojson(tiles_path, tiles_path + "predict_on_data/", tiles_path + "predictions_geo/")

Projecting 61 files


Projecting file 50 of 61: /content/drive/MyDrive/Sukute/tilespred_map10/predict_on_data/Prediction_odm_orthophoto_376753_3064085_45_20_32645.json


Combine predictions into full output

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
from detectree2.models.outputs import stitch_crowns, clean_crowns
#name = "Paracou"
folder = tiles_path+ "/predictions_geo"
crowns = stitch_crowns(folder, 1)


Stitching file 50 of 61: /content/drive/MyDrive/Sukute/tilespred_map10/predictions_geo/Prediction_odm_orthophoto_376753_3064085_45_20_32645.geojson


Remove invalid geometries before cleaning. Might be a way to fix them rather than remove all together but I think the ones with invalid geometries tend to be bad crowns anyway.

In [ ]:
crowns = crowns[crowns.is_valid]
# Simplify crowns to help with editing
crowns = crowns.set_geometry(crowns.simplify(0.3))
crowns

,Confidence_score,geometry
0,0.684301,"POLYGON ((311972.128 445336.473, 311970.128 44..."
1,0.665960,"POLYGON ((312006.128 445348.773, 312003.928 44..."
2,0.610145,"POLYGON ((311997.128 445333.073, 311993.528 44..."
3,0.569785,"POLYGON ((312016.628 445359.373, 312013.228 44..."
4,0.495516,"POLYGON ((311967.728 445355.473, 311965.928 44..."
...,...,...
7929,0.141906,"POLYGON ((312397.428 445289.473, 312394.128 44..."
7930,0.141382,"POLYGON ((312418.828 445298.973, 312417.728 44..."
7931,0.141144,"POLYGON ((312371.528 445320.573, 312369.528 44..."
7932,0.139175,"POLYGON ((312411.628 445318.273, 312408.128 44..."


Clean overlapping crowns. For significant overlaps, the most confident crown is selected. IoU can be varied based on how strict you want the filtering to be.

In [20]:
crowns = clean_crowns(crowns, 0.4)
crowns.to_file(site_path + "/crowns_map2.gpkg")

clearn_crowns: Performing spatial join...


clean_crowns: Processing candidate pairs: 100%|██████████| 11562/11562 [00:05<00:00, 1955.65it/s]
